# intracranial EEG preprocessing pipeline using MNE-python package

In [1]:
# Import relevant packages
import os
import numpy as np
import mne
import pandas as pd 



import matplotlib.pyplot as plt
import matplotlib
plt.ion()

In [2]:
# Define data file locations etc
homeDir = '/Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial'
os.chdir(homeDir)

# set montage file
#montage = homeDir + "montage/m.elp"
subjects = ['UCL01'] # Only analyzing 18-30 for now

filenames = ['clme.edf']

metadataL = ['1L_iEEG.csv']

metadataO = ['1O_iEEG.csv']

#version = []

#['S00_experimenter', 'S01', 'S02', 'S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S09', 'S10', 'S11', 'S12', 'S13', 'S14',
           #'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23']


In [4]:
# Load data
sub = 1 # 28 is in two files, need to fix -- Learning file for sub 30 fixed. 27,21, 19 has weird looking PSD
subInd = sub -1

raw_datafile = os.path.join(homeDir, subjects[subInd], filenames[subInd])
raw = mne.io.read_raw_edf(raw_datafile, preload=True, exclude = ['MKR1+', 'MKR2+'])#, exclude = ['xyz+', 'PULS+', 'BEAT+', 'SpO2', 'MKR2+', 'ches+', 'abd+', 'EEG EKG 1', 'EEG EKG 2', 'EEG EKG 3', 'EEG EKG4', 'MKR1+'])
#mne.io.set_eeg_reference(raw, ['RM'])

dfL = pd.read_csv(homeDir + '/' + str(subjects[subInd]) + '/'+ metadataL[subInd])
dfO = pd.read_csv(homeDir + '/' + str(subjects[subInd]) + '/'+ metadataO[subInd])
    
#raw = raw.interpolate_bads(reset_bads = False)

Extracting EDF parameters from /Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial/UCL01/clme.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3781119  =      0.000 ...  3692.499 secs...


In [5]:
# get channel names to exclude to only look at analog MKR channels
excludeChans = raw.info['ch_names']

rawAn = mne.io.read_raw_edf(raw_datafile, preload = True, exclude = excludeChans)

#save memory while extracting analog channels


Extracting EDF parameters from /Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial/UCL01/clme.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3781119  =      0.000 ...  3692.499 secs...


In [ ]:
# look at analog channels to get more precise timings of differen blocks, to later split up data to decrease RAM usage
%matplotlib notebook
# lala
fig = rawAn.plot(duration = 90)#, order = np.arange(0,125))
fig.set_figheight(10)
fig.set_figwidth(10)

In [6]:
# extract onsets and offsets of analog channel triggers

ana = rawAn._data[1,:] - rawAn._data[0,:]

print(np.mean(ana))
print(np.max(ana))
print(np.std(ana))
sd = np.std(ana)
ana = (ana - np.mean(ana))
anaFilt = ana.copy()
#rawAn._data = np.concatenate((rawAn._data, ana.T))
#rawAn.info['ch_names'].append('convertAna')
#rawAn._data[0,:] = ana

onset = 0
eventOnset = []
eventOffset = []

#ana = np.convolve(ana, np.ones((20,))/20, mode='valid') # clean up signal -- note: may need to check for significant time-shifts
threshold = 0.00001
#rawAn._data = np.concatenate((rawAn._data, ana))
#rawAn.info['ch_names'].append('filtAna')
#rawAn._data[1,:] = ana
winSize = 50

for i in range(winSize,len(ana) - winSize):
    
    # if analog signal is above the threshold, and no onset has been recorded
    if np.abs(ana[i]) > threshold and onset == 0:
        # Record onset if this is a sustained threshold crossing (avoing blips to be recorded as events)
        if np.sum(np.abs(ana[(i-winSize):i]) < threshold) > 40 and np.sum(np.abs(ana[i:(i+winSize)] )> threshold) >15: # record onset only if previous 50 samples were below threshold
            onset = 1
            eventOnset.append(i)
            
    # if an onset has been recorded detect when the analog channel goes back to baseline
    elif np.abs(ana[i]) < threshold and onset == 1:
        if np.sum(np.abs(ana[i:(i+winSize)] )<threshold) > 40 and np.sum(np.abs(ana[(i-winSize):i]) > threshold) >10:
            onset = 0
            eventOffset.append(i)

print('counted ' + str(len(eventOnset)) + ' event onsets')
print('counted ' + str(len(eventOffset)) + ' event offsets')

-2.4865192688542025e-06
5.000076295109483e-05
1.7597496229652853e-05
counted 529 event onsets
counted 529 event offsets


In [ ]:
# plot analog channel with event onset and offsets

# create events array 
%matplotlib notebook

events = np.zeros((len(eventOnset) + len(eventOffset), 3))
events[:,0] = np.concatenate((np.asarray(eventOnset), np.asarray(eventOffset)))
events[0:len(eventOnset), 2] = 1.0
                             

fig = rawAn.plot(duration = 15, events = events)
fig.set_figheight(7)
fig.set_figwidth(10)
#for i in eventOnset:
    
    #plt.axvline(x=i, color = 'gray')



In [7]:
%matplotlib qt
eventDur = np.asarray(eventOffset) - np.asarray(eventOnset)
plt.hist(eventDur, bins = 100) #, bins = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500])

(array([  1.,   0.,   2.,   2., 207.,   3.,  14.,  10.,  25.,   7.,   0.,
         35.,  10.,   0.,  34.,  58.,   0.,   1.,   2.,   9.,   1.,  41.,
          1.,   1.,   0.,   0.,   0.,   0.,   5.,   0.,   0.,  27.,  15.,
          1.,   0.,   1.,   0.,   0.,   0.,   1.,   1.,   0.,   0.,   0.,
          0.,   1.,   0.,   0.,   0.,   0.,   1.,   1.,   2.,   0.,   0.,
          1.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
          0.,   1.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   1.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,
          1.]),
 array([  10.  ,   26.68,   43.36,   60.04,   76.72,   93.4 ,  110.08,
         126.76,  143.44,  160.12,  176.8 ,  193.48,  210.16,  226.84,
         243.52,  260.2 ,  276.88,  293.56,  310.24,  326.92,  343.6 ,
         360.28,  376.96,  393.64,  410.32,  427.  ,  443.68,  460.36,
         477.04,  493.72,  510.4 ,

In [ ]:
# plot analog channel with event onset and offsets

# create events array 
%matplotlib notebook

events = np.zeros((len(eventsLBeep) + len(eventsOBeep), 3))
events[:,0] = np.concatenate((np.asarray(eventsLBeep), np.asarray(eventsOBeep)))
#events[0:len(eventsOBeep), 2] = 1.0
                             

fig = rawAn.plot(duration = 15, events = events)
fig.set_figheight(3)
fig.set_figwidth(10)

In [8]:
# exclude events outside blocks (i.e. practice and test sounds)

sfreq = rawAn.info['sfreq']
Ltimes = [702, 1920] 
Otimes = [2390, 2990] 

# convert to timestamps
LInd = [0,0]
OInd = [0,0]

LInd[0] = np.round(sfreq*Ltimes[0])
LInd[1] = np.round(sfreq*Ltimes[1])
OInd[0] = np.round(sfreq*Otimes[0])
OInd[1] = np.round(sfreq*Otimes[1])

eventsL = np.zeros((len(eventOnset),3))
eventsO = np.zeros((len(eventOnset),3))
eventsLBeep = []
eventsOBeep = []

Li = 0
Oi = 0
prevStamp = 0

for tstamp, tdur in zip(np.asarray(eventOnset), np.asarray(eventDur)):
    if tstamp < LInd[1] and tstamp > LInd[0]:
        interval = (tstamp-prevStamp)/sfreq
        eventsL[Li, :] = np.asarray([tstamp, tdur, interval])
        if tdur > 59 and tdur < 105:
            eventsLBeep.append(tstamp) # first trial skipped
            #print(tstamp/sfreq)
            #print(tdur)
        Li += 1
        prevStamp = tstamp
    elif tstamp <OInd[1] and tstamp > OInd[0]:
        interval = (tstamp-prevStamp)/sfreq
        eventsO[Oi,:] = np.asarray([tstamp, tdur, interval])
        if tdur > 81 and tdur <105:
            eventsOBeep.append(tstamp)
            #print(tstamp/sfreq)
            #print(tdur)
            
            
        prevStamp = tstamp
        Oi += 1
        
print(eventsL.shape)
eventsO
print(Li)
print(Oi)

# add missing trials
learnMissed = np.asarray([689.981, 1231.03, 1219.0, 1398.97, 1517.18, 1524.9, 1650.12, 1690.52]) # remove these trials later
# convert to tstamp
learnMissed = learnMissed*sfreq
# add to eventsLBeep
for i in learnMissed: eventsLBeep.append(round(i))
# sort eventsLBeep
eventsLBeep.sort()


OMissed = np.asarray([2818.32, 2915.29, 2923.17, 2972.74])
OMissed = OMissed*sfreq
for i in OMissed: eventsOBeep.append(round(i))
eventsOBeep.sort()

(529, 3)
271
172


In [9]:
# check timings with logfiles

trialDiff = []


for i in range(1, len(eventsLBeep)-1):
    trialDiff.append( (eventsLBeep[i] - eventsLBeep[i-1])/sfreq)

trialDiffLog = []
trialDiff.append(0.0)
for i in range(1, len(dfL) ):
    trialDiffLog.append((dfL['trialStamp'][i] - dfL['trialStamp'][i-1])/1000)
trialDiffLog.append(0.3)
timings = pd.DataFrame(data = {'log': trialDiffLog, 'analog': trialDiff})

# add this info to dfL, now it's the time diff between that trial and the next trial. If not the same within a margin, remove that trial
# double check this with timestamps
dfL['logDiff'] = trialDiffLog
dfL['analogDiff'] = trialDiff

timings

,log,analog
0,14.0488,12.648438
1,15.5320,15.532227
2,14.4821,14.492188
3,12.4990,12.491211
4,12.1989,12.202148
5,13.4489,13.451172
6,13.5156,13.522461
7,13.2322,13.231445
8,12.2323,12.231445
9,11.4157,11.420898


In [10]:
# check timings with logfiles
del trialDiff, trialDiffLog
trialDiff = []


for i in range(1, len(eventsOBeep)):
    trialDiff.append( (eventsOBeep[i] - eventsOBeep[i-1])/sfreq)

trialDiffLog = []
trialDiff.append(0.0)
trialDiff.append(0.0)
for i in range(1, len(dfO) ):
    trialDiffLog.append((dfO['trialStamp'][i] - dfO['trialStamp'][i-1])/1000)
trialDiffLog.append(0.3)
timings = pd.DataFrame(data = {'log': trialDiffLog, 'analog': trialDiff})

dfO['logDiff'] = trialDiffLog
dfO['analogDiff'] = trialDiff
timings

,log,analog
0,7.7660,6.395508
1,5.7662,5.760742
2,5.2829,5.291016
3,6.2994,6.300781
4,6.6495,6.650391
5,5.4828,5.480469
6,5.2829,5.281250
7,5.5496,5.550781
8,5.0495,5.049805
9,6.3995,6.401367


In [11]:
eventsL = np.zeros(( len(eventsLBeep), 3))
eventsL[:,0] = eventsLBeep
eventsL[:,2] = np.ones((len(eventsLBeep)))

eventsO = np.zeros(( len(eventsOBeep), 3))
eventsO[:,0] = eventsOBeep
eventsO[:,2] =2.0*np.ones((len(eventsOBeep)))

events = np.concatenate((eventsL, eventsO), axis = 0)
events.astype(int)
df = dfL.append(dfO, ignore_index = True, sort = False)

In [ ]:
# Check power spectrum
raw.plot_psd(fmax = 140)

## Filter data

### Required: filter and downsample data

In [12]:

raw.filter(0.3, 250,  fir_design="firwin")
#raw, events = raw.resample(100, npad='auto', events=events)

#raw.drop_channels(["MKR1+", "MKR2+"])

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 2.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 250.00 Hz
- Upper transition bandwidth: 62.50 Hz (-6 dB cutoff frequency: 281.25 Hz)
- Filter length: 11265 samples (11.001 sec)



<RawEDF  |  clme.edf, n_channels x n_times : 130 x 3781120 (3692.5 sec), ~3.66 GB, data loaded>

#### Optional: look at PSD after filtering|

In [ ]:
%matplotlib inline
raw.plot_psd(fmax = 40)

In [ ]:
events

In [13]:
# Remove eye-artifact ICA's from raw data, re-epoch
#del epochsEye # Clear memory

tmin = -3.0
tmax = 12.0 # make it long enough sufficient for TFR analyses




keys     = ['Learning', 'Orienting']

values   = [ 1, 2]

event_ids_target = dict(zip(keys, values))

# Remove bad ica

#raw2 = ica.apply(raw, exclude=ica_exclude).pick_types(eeg=True)
#raw2 = raw2.interpolate_bads(reset_bads=True, verbose=False)

# Epoch data

epochs = mne.Epochs(raw, events.astype(int), event_ids_target, tmin=tmin, tmax=tmax, preload=True, metadata = df) # set reject to None because we will do autoreject

epochs.events[:,1] = range(1,len(epochs.events)+1) #Fill that weird zeros column with the trial numbers

epochs.apply_baseline((None, 0))



192 matching events found
Applying baseline correction (mode: mean)
Adding metadata with 48 columns
0 projection items activated
Loading data for 192 events and 15361 original time points ...
0 bad epochs dropped
Applying baseline correction (mode: mean)


<Epochs  |   192 events (all good), -3 - 12 sec, baseline [None, 0], ~2.86 GB, data loaded, with metadata,
 'Learning': 97
 'Orienting': 95>

In [14]:
# Remove trials with clear spikes in HPC, select so trial turns red, hit 'power' button at the end and trials will be removed from epochs
chans = ['EEG aH', 'EEG pH'] # select which probes to look at
elecs = []
for i in range(len(epochs.info['ch_names'])): 
    for k in chans: 
        if k in epochs.info['ch_names'][i]:
            
            elecs.append(i)

fig = epochs.plot(n_epochs = 3, picks = elecs, n_channels = 30)
fig.set_figheight(8)
fig.set_figwidth(10)

Dropped 41 epochs: 0, 10, 20, 36, 38, 39, 40, 41, 42, 48, 49, 53, 54, 58, 59, 62, 63, 72, 75, 76, 83, 92, 93, 94, 95, 99, 102, 109, 112, 115, 130, 134, 135, 142, 144, 146, 147, 152, 155, 160, 189
Channels marked as bad: []


In [22]:
epochs.metadata['RTplace1']

1      1167.1
2      1289.1
3       999.7
4      1066.7
5      1262.4
6      1544.9
7      1032.0
8      1234.7
9      1010.8
11      800.4
12      940.8
13     1342.6
14      855.0
15      996.7
16      982.7
17      994.4
18      927.4
19     1127.9
21      815.3
22      848.2
23     1027.3
24     1071.0
25      876.0
26      990.9
27     1256.7
28     1065.0
29     1048.1
30      860.5
31      811.4
32     1184.5
        ...  
161       NaN
162       NaN
163       NaN
164       NaN
165       NaN
166       NaN
167       NaN
168       NaN
169       NaN
170       NaN
171       NaN
172       NaN
173       NaN
174       NaN
175       NaN
176       NaN
177       NaN
178       NaN
179       NaN
180       NaN
181       NaN
182       NaN
183       NaN
184       NaN
185       NaN
186       NaN
187       NaN
188       NaN
190       NaN
191       NaN
Name: RTplace1, Length: 151, dtype: float64

In [ ]:
# Remove bad channels -- do manually based on above plots -- write code to reject bad channels based on variance, use sub 30 as example
%matplotlib qt
%matplotlib qt
raw.plot()

In [29]:
# Remove outlier trials -- can only run once, otherwise you start to remove more trials!

# Drop eye movement trials
#epochs.drop(eyeArtifactInd)

axis, keep_idx = plot_AR(epochs, method='gesd')



ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
epochs = epochs[keep_idx[0]]

In [16]:
# Save ERP data
fileName = 'dataClean-epo.fif'
saveFile = os.path.join(homeDir, subjects[subInd], fileName)
epochs.save(saveFile, overwrite = True, verbose = True)



Overwriting existing file.


In [36]:
saveFile

'/Users/yvonnefonken/Documents/Oxford/Research Projects/intracranial/UCL01/dataClean-epo.fif'

In [ ]:
del epochsEye
del raw
del raw2

In [ ]:
epochs.info['ch_names'] 

## Do TFR  -- scratch this for now. Saving data suitable for TFR and ERP analyses instead. 

In [ ]:
# Re-load and re-filter data
subInd = sub -1

raw_datafile = os.path.join(homeDir, subjects[subInd], filenames[subInd])
rawTFR = mne.io.read_raw_eeglab(raw_datafile, montage, eog=['VEOG', 'HEOG'], preload=True)
mne.io.set_eeg_reference(rawTFR, ['RM'])

# Reload events (which are downsampled earlier)

# Housekeeping, filtering, and downsampling
    
# Find the first even with a real trigger code (this is because of the impedence trigger)
#f = np.min(np.where(events[:,-1] < 256))

# crop 10 seconds before the first event
tmin = rawTFR.times[eventsTFR[0][0,0]]-10
if tmin < 0: # if tmin is negative just set it at the beginning
    tmin = rawTFR.times[0] 
# crop 10 seconds after the last event
tmax = rawTFR.times[eventsTFR[0][-1,0]]+10
if tmax > rawTFR.times[-1]: # if tmax is bigger then the max time, just take that
    tmax = rawTFR.times[-1]

# crop it
rawTFR.crop(tmin=tmin, tmax=tmax)

rawTFR.filter(lpTF, hpTF,  fir_design="firwin") 
#rawTFR, events = rawTFR.resample(sfreq, npad='auto', events=events[0]) # Resample later

rawTFR.drop_channels(["RM"])





In [ ]:
# Epoch and Remove ica and bad trials

# Remove eye-artifact ICA's from raw data, re-epoch
#del epochsEye # Clear memory

tmin = -2.0
tmax = 3.0




keys     = ['Blimp']

values   = [ 50]

event_ids_target = dict(zip(keys, values))

# Remove bad ica

rawTFR2 = ica.apply(rawTFR, exclude=ica.exclude).pick_types(eeg=True)

# Epoch data

epochsTFR = mne.Epochs(rawTFR2, eventsTFR[0], event_ids_target, tmin=tmin, tmax=tmax, preload=True, metadata = df) # set reject to None because we will do autoreject

epochsTFR.events[:,1] = range(1,len(epochsTFR.events)+1) #Fill that weird zeros column with the trial numbers

epochsTFR.apply_baseline((None, 0))

# drop eye movement excluded trials
epochsTFR.drop(eyeArtifactInd)
# drop high variance trials
epochsTFR = epochsTFR[keep_idx[0]] # check whether this also updates df -- checked, it does


In [ ]:
# Implement TFR magic
frequencies = np.logspace(*np.log10([3, 45]), num = 15)  # define frequencies of interest
n_cycles = frequencies / 5.  # different number of cycle per frequency
Fs = rawTFR.info['sfreq']  # sampling in Hz
decim = 10
data = epochsTFR.get_data()
print(data.shape)

from mne.time_frequency import tfr_morlet
power = tfr_morlet(epochsTFR, freqs=frequencies,n_cycles=n_cycles, use_fft=False, decim=decim, zero_mean=True, average = True, return_itc = False)
print(power._data.shape)
print ('done!')


In [ ]:
power._data.shape

In [ ]:
# Check data

power.plot_topomap(ch_type='EEG', tmin=0.6, tmax=1.0, fmin=8, fmax=12,
                   baseline=(-0.5, 0), mode='logratio', axes=axis[1],
                   title='Alpha')

power.plot_topomap(ch_type='EEG', tmin=0.8, tmax=1.5, fmin=13, fmax=30,
                   baseline=(-0.5, 0), mode='logratio', axes=axis[1],
                   title='Beta')

In [ ]:
# Save TFR 
fileName = 'testTFR-epo.fif'
saveFile = os.path.join(homeDir, subjects[subInd], fileName)
power.save(saveFile, overwrite = False, verbose = True)

In [20]:
def gesd(x, alpha = 0.05, p_out = .1, outlier_side = 0):
    import numpy as np
    import scipy.stats
    import copy
    
    '''
    Detect outliers using Generalizes ESD test
    based on the code from Romesh Abeysuriya implementation for OSL
      
    Inputs:
    - x : Data set containing outliers - should be a np.array 
    - alpha : Significance level to detect at (default = .05)
    - p_out : percent of max number of outliers to detect (default = 10% of data set)
    - outlier_side : Specify sidedness of the test
        - outlier_side = -1 -> outliers are all smaller
        - outlier_side = 0 -> outliers could be small/negative or large/positive (default)
        - outlier_side = 1 -> outliers are all larger
        
    Outputs
    - idx : Logicial array with True wherever a sample is an outlier
    - x2 : input array with outliers removed
    
    For details about the method, see
    B. Rosner (1983). Percentage Points for a Generalized ESD Many-outlier Procedure, Technometrics 25(2), pp. 165-172.
    http://www.jstor.org/stable/1268549?seq=1
    '''

    if outlier_side == 0:
        alpha = alpha/2
    
    
    if type(x) != np.ndarray:
        x = np.asarray(x)

    n_out = int(np.ceil(len(x)*p_out))

    if any(~np.isfinite(x)):
        #Need to find outliers only in non-finite x
        y = np.where(np.isfinite(x))[0] # these are the indexes of x that are finite
        idx1, x2 = gesd(x[np.isfinite(x)], alpha, n_out, outlier_side)
        # idx1 has the indexes of y which were marked as outliers
        # the value of y contains the corresponding indexes of x that are outliers
        idx = [False] * len(x)
        idx[y[idx1]] = True

    n      = len(x)
    temp   = x
    R      = np.zeros((1, n_out))[0]
    rm_idx = copy.deepcopy(R)
    lam    = copy.deepcopy(R)

    for j in range(0,int(n_out)):
        i = j+1
        if outlier_side == -1:
            rm_idx[j] = np.nanargmin(temp)
            sample    = np.nanmin(temp)
            R[j]      = np.nanmean(temp) - sample
        elif outlier_side == 0:
            rm_idx[j] = int(np.nanargmax(abs(temp-np.nanmean(temp))))
            R[j]      = np.nanmax(abs(temp-np.nanmean(temp)))
        elif outlier_side == 1: 
            rm_idx[j] = np.nanargmax(temp)
            sample    = np.nanmax(temp)
            R[j]      = sample - np.nanmean(temp)
        
        R[j] = R[j] / np.nanstd(temp)
        temp[int(rm_idx[j])] = np.nan
        
        p = 1-alpha/(n-i+1)
        t = scipy.stats.t.ppf(p,n-i-1)
        lam[j] = ((n-i) * t) / (np.sqrt((n-i-1+t**2)*(n-i+1)))
    
    #And return a logical array of outliers
    idx = np.zeros((1,n))[0]
    idx[np.asarray(rm_idx[range(0,np.max(np.where(R>lam))+1)],int)] = np.nan
    idx = ~np.isfinite(idx)
    
    x2 = x[~idx]

        
    return idx, x2



In [21]:
def plot_AR(epochs, method = 'gesd', zthreshold = 1.5, p_out = .1, alpha = .05, outlier_side = 1):
    import seaborn as sns
    import pandas as pd
    import scipy.stats

    #Get data, variance, number of trials, and number of channels
    dat     = epochs.get_data()
    var     = np.var(dat, 2)
    ntrials = np.shape(dat)[0]
    nchan   = len(epochs.ch_names)

    #set up the axis for the plots
    x_epos  = range(1,ntrials+1)
    y_epos  = np.mean(var,1)
    y_chans = range(1,nchan+1)
    x_chans = np.mean(var,0)

    #scale the variances
    y_epos  = [x * 10**6 for x in y_epos]
    x_chans = [x * 10**6 for x in x_chans]

    #Get the zScore
    zVar = scipy.stats.zscore(y_epos)

    #save everything in the dataFrame
    df_epos           = pd.DataFrame({'var': y_epos, 'epochs': x_epos, 'zVar': zVar})
    df_chans          = pd.DataFrame({'var': x_chans, 'chans': y_chans})
    
    # Apply the artefact rejection method
    if method == 'gesd':
        idx,x2            = gesd(y_epos, p_out=p_out, alpha=alpha, outlier_side=outlier_side) #use the gesd to find outliers (idx is the index of the outlier trials)
        keepTrials        = np.ones((1,ntrials))[0]
        keepTrials[idx]   = 0
        title = 'Generalized ESD test (alpha=' + str(alpha) + ', p_out=' + str(p_out) + ', outlier_side=' + str(outlier_side) + ')'
    elif method == 'zScore':
        keepTrials        = np.where(df_epos['zVar'] > zthreshold, 0, 1)
        title = 'ZVarience threshold of ' + str(zthreshold)
    elif method == 'none':
        title = 'no additional artefact rejection '
        keepTrials        = np.ones((1,ntrials))[0]
    
    df_epos['keepTrial'] = keepTrials
    df_keeps = df_epos[df_epos['keepTrial'] == 1]
    print(str(ntrials - len(df_keeps)) + ' trials discarded')
    
    # get the clean data
    keep_idx    = np.asarray(np.where(keepTrials),int)
    clean_dat    = np.squeeze(dat[keep_idx])
    
    #recalculate the var for chan
    clean_var    = np.var(clean_dat, 2)
    x_chans_c    = np.mean(clean_var,0)
    x_chans_c    = [x * 10**6 for x in x_chans_c]

    df_chans_c   = pd.DataFrame({'var': x_chans_c, 'chans': y_chans})
    
    
    # Plot everything
    fig, axis = plt.subplots(2, 2, figsize=(12, 12))
    axis[0,0].set_ylim([0, max(y_epos) + min(y_epos)*2])
    axis[0,1].set_xlim([0, max(x_chans)+ min(x_chans)*2])
    axis[1,0].set_ylim([0, max(df_keeps['var'])+ min(df_keeps['var'])*2])
    axis[1,1].set_xlim([0, max(x_chans_c)+ min(x_chans_c)*2])

    axis[0,0].set_title(title)
    sns.scatterplot(x = 'epochs', y = 'var', hue = 'keepTrial', hue_order = [1,0], ax = axis[0,0], data = df_epos)
    sns.scatterplot(x = 'var', y = 'chans', ax = axis[0,1], data = df_chans)
    sns.scatterplot(x = 'epochs', y = 'var', ax = axis[1,0], data =df_keeps)
    sns.scatterplot(x = 'var', y = 'chans', ax = axis[1,1], data = df_chans_c)
    
    
    
    return axis, keep_idx

